In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_json('../input/train.json')

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(40)
X = train[['longitude', 'latitude']]
kmeans.fit(X)

plt.scatter(kmeans.cluster_centers_.T[0], kmeans.cluster_centers_.T[1], alpha=1, c='b')
plt.scatter(train.longitude.values, train.latitude.values, alpha=.01, c='r')
# plt.xlim([-74.25, -73.5])
# plt.ylim([40.4, 41])
# plt.xlim([-74.05, -73.9])
# plt.ylim([40.65, 40.9])
plt.xlim([-74.25, -73.6])
plt.ylim([40.55, 40.9])

In [ ]:
def get_percent_caps(descrip):
    CAPS = [chr(65+x) for x in range(26)]
    LOWERS = [chr(97+x) for x in range(26)]
    descrip = [x for x in descrip if x in CAPS or x in LOWERS]
    ncaps = sum([x in CAPS for x in descrip])
    return ncaps * 1. / len(descrip) if len(descrip) > 0 else 0.

def get_percent_exclam(descrip):
    CAPS = [chr(65+x) for x in range(26)]
    LOWERS = [chr(97+x) for x in range(26)]
    EXTRAS = ['!','?','.',',',';']
    descrip = ''.join([x for x in descrip if x in CAPS or x in LOWERS or x in EXTRAS])
    ncaps = sum([x == '!' for x in descrip])
    return ncaps * 1. / len(descrip) if len(descrip) > 0 else 0.

In [ ]:
[x for x in aptFeatures if 'flex' in x.lower()]

In [ ]:
train['LaundryInBuilding'] = [0+any(['laundry' in x.lower() and 'in unit' not in x.lower() and 'in-unit' not in x.lower() for x in y]) for y in train.features]
train['LaundryInUnit'] = [0+any([('laundry' in x.lower() or 'washer' in x.lower()) and ('in unit' in x.lower() or 'in-unit' in x.lower()) for x in y]) for y in train.features]
train['NoFee'] = [0+any(['no fee' in x.lower() or 'no-fee' in x.lower() for x in y]) for y in train.features]
train['Outside'] = [0+any([any([z in x.lower() for z in ('outdoor', 'common outdoor space', 'balcony', 'deck', 'terrace', 'garden', 'patio')]) for x in y]) for y in train.features]
train['Elevator'] = [0+any(['elevator' in x.lower() for x in y]) for y in train.features]
train['Gym'] = [0+any(['fitness center' in x.lower() or 'gym' in x.lower() for x in y]) for y in train.features]
train['Cats'] = [0+any(['cats allowed' in x.lower() or 'cats on approval' in x.lower() for x in y]) for y in train.features]
train['Dogs'] = [0+any(['dogs allowed' in x.lower() or 'dogs on approval' in x.lower() for x in y]) for y in train.features]
train['PreWar'] = [0+any(['pre-war' in x.lower() or 'pre war' in x.lower() for x in y]) for y in train.features]
train['NewConstruction'] = [0+any(['new construction' in x.lower() for x in y]) for y in train.features]
train['Furnished'] = [0+any(['furnished' in x.lower() for x in y]) for y in train.features]
train['Renovated'] = [0+any(['renovated' in x.lower() for x in y]) for y in train.features]
train['ReducedFee'] = [0+any(['reduced fee' in x.lower() or 'reduced-fee' in x.lower() for x in y]) for y in train.features]
train['Garage'] = [0+any(['garage' in x.lower() or 'parkig' in x.lower() for x in y]) for y in train.features]
train['Doorman'] = [0+any(['doorman' in x.lower() for x in y]) for y in train.features]
train['Exclusive'] = [0+any(['exclusive' in x.lower() for x in y]) for y in train.features]
train['LengthDescription'] = [len(x) for x in train.description]
train['PercentCAPS'] = [get_percent_caps(x) for x in train.description]
train['Percent!'] = [get_percent_exclam(x) for x in train.description]
train['Flex'] = [0+any(['flex' in x and 'flexible' not in x for x in y]) for y in train.features]
train['nPhotos'] = train.photos.apply(len)
train["created"] = pd.to_datetime(train["created"])
train["created_year"] = train["created"].dt.year
train["created_month"] = train["created"].dt.month
train["created_day"] = train["created"].dt.day

In [ ]:
nbedrooms = set(train.bedrooms)

In [ ]:
trainingSets = dict()
for nBedrooms in nbedrooms:
    trainingSets[nBedrooms] = train[train.bedrooms == nBedrooms]

In [ ]:
d = {}
for nBedrooms in nbedrooms:
    nTrain = trainingSets[nBedrooms]
    lo = (nTrain.interest_level=='low').mean()
    md = (nTrain.interest_level=='medium').mean()
    hi = (nTrain.interest_level=='high').mean()
    d[nBedrooms] = dict(lo=lo, mid=md, hi=hi, listings=len(nTrain))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

In [ ]:
cols = [x for x in train.columns if x not in
        ['building_id', 'created', 'description', 'display_address', 'features',
         'interest_level', 'latitude', 'listing_id', 'longitude', 'manager_id',
         'photos', 'street_address', 'created']]
classifiers = dict()
Xs = dict()
Ys = dict()
for n in nbedrooms:
    X = train[cols]
    y = train.interest_level
    X_train, X_test, Y_train, Y_test = train_test_split(X, y)
    clf = RandomForestClassifier(100)
    clf.fit(X_train, Y_train)
    classifiers[n] = clf
    Xs[n] = dict(train=X_train, test=X_test)
    Ys[n] = dict(train=Y_train, test=Y_test)

In [ ]:
big_test = []
big_probs = []
for n in [4]:#nbedrooms:
    X_test = Xs[n]['test']
    Y_test = Ys[n]['test']
    X_train = Xs[n]['train']
    Y_train = Ys[n]['train']
    clf = classifiers[n]

    #Y_pred = clf.predict(X_test)
    Y_prob = clf.predict_proba(X_test)
    big_test.extend(Y_test)
    big_probs.extend(Y_prob)

In [ ]:
log_loss(big_test, big_probs)

In [ ]:
X = X_train[['bathrooms', 'bedrooms', 'LaundryInBuilding', 'LaundryInUnit',
       'NoFee', 'Outside', 'Elevator', 'Gym', 'Cats', 'Dogs', 'PreWar',
       'NewConstruction', 'Furnished', 'Renovated', 'ReducedFee', 'Garage',
       'Doorman']]
X['const'] = 1
Y = X_train.price

In [ ]:
beta = np.linalg.solve(X.T.dot(X), X.T.dot(Y))

In [ ]:
pd.Series(beta, index=X.columns)

In [ ]:
X = X_test[['bathrooms', 'bedrooms', 'LaundryInBuilding', 'LaundryInUnit',
       'NoFee', 'Outside', 'Elevator', 'Gym', 'Cats', 'Dogs', 'PreWar',
       'NewConstruction', 'Furnished', 'Renovated', 'ReducedFee', 'Garage',
       'Doorman']]
X['const'] = 1
Y_test = X_test['price']

In [ ]:
y_pred = X.dot(beta)

In [ ]:
plt.scatter(Y_test, y_pred)
plt.xlim([0, 30000])
plt.ylim([-1000, 30000])

In [ ]:
numY = [dict(low=0, medium=1, high=2).get(x) for x in Y_train]
v = X_train.copy()
v['interest'] = numY
v.corr().iloc[-1]